# Reconhecer rostos em vídeo ao vivo usando sua webcam - Simples

Este é um exemplo super simples (mas lento) de executar o reconhecimento facial em vídeo ao vivo da sua webcam.
Há um segundo exemplo que é um pouco mais complicado, mas roda mais rápido.

In [10]:
import requests
import numpy as np
import cv2
from deepface import DeepFace
import face_recognition
from io import BytesIO
from PIL import Image

In [11]:
# url de acesso da emotions-api
base_url = 'http://localhost:3333'

In [12]:
# Realiza uma solicitação GET para obter a lista de usuários
response = requests.get(f'{base_url}/user/list')
users = response.json()
users

[{'id': '3c831627-d1ae-46ae-99d6-8bc1ca85e000',
  'name': 'Lourival Oliveira Melo',
  'avatar_url': 'http://localhost:3333/uploads/aa2803df0511-perfil-papai.jpeg'},
 {'id': 'e42ca150-ecc2-4102-b923-917ed7b1b60c',
  'name': 'Leonaro Nunes Conçalves',
  'avatar_url': 'http://localhost:3333/uploads/047b69e79667-perfil-leo.jpeg'},
 {'id': 'eef2855f-dfa0-4c28-a6d7-fa1ba4257d43',
  'name': 'Igor Melo Garcia',
  'avatar_url': 'http://localhost:3333/uploads/fcf67e515a71-perfil-igor.jpeg'},
 {'id': '364390ee-d9fb-43ef-bbfd-6a91649beff1',
  'name': 'Miguel Ângelo Mocbel',
  'avatar_url': 'http://localhost:3333/uploads/2e21ccb770eb-perfil-miguel.jpeg'},
 {'id': '012445df-5f0b-4957-8a3d-8bcfd5441276',
  'name': 'Laciene Melo Garcia',
  'avatar_url': 'http://localhost:3333/uploads/1c49fe35e4f6-IMG_20211014_090554.jpg'}]

In [13]:
# Inicializa uma lista de objetos contendo codificações faciais e nomes correspondentes
known_faces = []
# Itera sobre a lista de usuários

for user in users:
    # carrega a imagem do usuário
    user_image_url = user['avatar_url']

    # baixa a imagem usando requests
    response_image = requests.get(user_image_url)
    image_data = BytesIO(response_image.content)

    # converte a imagem para um formato suportado pelo face-recognition
    pil_image = Image.open(image_data).convert("RGB")
    user_image = np.array(pil_image)

    # realiza o processo de reconhecimento facial
    user_face_encoding = face_recognition.face_encodings(user_image)[0]

    # adiciona a codificação facial o nome à lista como um objeto
    known_faces.append({'id': user['id'], 'encoding': user_face_encoding, 'name': user['name']})



In [14]:
# Inicializando a webcam
video_capture = cv2.VideoCapture(0)

In [15]:
while True:
    # Capturando um frame do vídeo
    ret, frame = video_capture.read()

    # encontrando todas as faces no frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # verificando se a face é conhecida
        matches = face_recognition.compare_faces([user['encoding'] for user in known_faces], face_encoding)

        name = "Desconhecido"

        # se uma correspondência for encontrada, use o nome correspondente
        if True in matches:
            first_match_index = matches.index(True)
            name = known_faces[first_match_index]['name']

        # Desenhando um retângulo ao redor do rosto
        cv2.rectangle(frame, pt1=(left, top), pt2=(right, bottom), color=(0, 255, 0), thickness=2)

        # exibindo o nome da pessoa identificada
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, f"User: {name}", (left + 6, bottom + 40), font, 0.5, (255, 255, 255), 1)

    # exibindo o frame resultante
    cv2.imshow('Video', frame)

    # saindo do loop quando a tecla 'q' é pressionada
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberando os recursos
video_capture.release()
cv2.destroyAllWindows()